In [ ]:
%reset

# 1. Imports

In [74]:
import mne
import time
import os
import custom_modules.xdf_to_fif_converter as xtfc
import custom_modules.preprocessing_ptu as prep
import custom_modules.file_handling as  fh

import importlib
importlib.reload(xtfc)
importlib.reload(prep)
importlib.reload(fh)

<module 'custom_modules.file_handling' from 'C:\\Users\\peter\\Documents\\Code\\master-thesis\\03_analysis\\custom_modules\\file_handling.py'>

# 2. Constants

In [69]:
data_path = 'D:/Diplomarbeit_data/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']
trial_type_markers = ['LTR-s', 'LTR-l','RTL-s', 'RTL-l', 'TTB-s', 'TTB-l', 'BTT-s', 'BTT-l']

plot = False
mne.set_log_level('WARNING')

# 3. XDF --> FIF
The following cell reads all xdf-files containing the raw eeg and loads them into the mne structure and stores them as .fif-files.

In [70]:
help(xtfc.xdf_to_fif)

Help on function xdf_to_fif in module custom_modules.xdf_to_fif_converter:

xdf_to_fif(src, dst, sbj)
    Convert .xdf files to .fif files for a given subject.
    
    :param src: str, path of the folder containing .xdf files.
    :param dst: str, path of the folder where to save the .fif files.
    :param sbj: str, subject identifier.



In [71]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='raw', dst_fldr='raw_fif')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Extracting subject {subject}', end=' ')

    # Call the xdf_to_fif function from the xtfc module which takes care of converting the xdf files to fif files:
    xtfc.xdf_to_fif(src_path, dst_path, subject)

    print()

print(f'Finished reading, took me {round(time.time()-start)} seconds...')

Extracting subject A01 # # # # # # # # # # # 
Extracting subject A02 # # # # # # # # # # # 
Extracting subject A03 # # # # # # # # # # # 
Extracting subject A04 # # # # # # # # # # # 
Extracting subject A05 # # # # # # # # # # # 
Extracting subject A06 # # # # # # # # # # # 
Extracting subject A07 # # # # # # # # # # # 
Extracting subject A08 # # # # # # # # # # # 
Extracting subject A09 # # # # # # # # # # # 
Extracting subject A10 # # # # # # # # # # # 
Finished reading, took me 75 seconds...


# 4. Concatenate all fif files for each subject

In [72]:
help(prep.concat_fifs)

Help on function concat_fifs in module custom_modules.preprocessing_ptu:

concat_fifs(src, dst, sbj, paradigm='paradigm')
    Concatenates multiple raw.fif files from a single subject and paradigm.
    
    :param src: str, path to the directory containing the input raw.fif files.
    :param dst: str, path to the directory where the concatenated raw.fif file will be stored.
    :param sbj: str, subject name contained in the file names.
    :param paradigm: str, paradigm name contained in the file names (default is 'paradigm').
    :return: None
    
    - Searches for all the files in the source directory containing the subject name, 'raw.fif' and the specified paradigm.
    - Reads each of the selected raw.fif files using MNE-Python's read_raw function.
    - Concatenates the read files into a single raw object using MNE-Python's concatenate_raws function.
    - Saves the concatenated raw object as a new raw.fif file in the destination directory.



In [73]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='raw_fif', dst_fldr='concat')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Reading all fif files for subject {subject}', end=' ')

    # Concatenate all raw files for the paradigm and the eye paradigm:
    prep.concat_fifs(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')
    prep.concat_fifs(src=src_path, dst=dst_path, sbj=subject, paradigm='eye')
    print()

print(f'Finished concatenating, took me {round(time.time() - start)} seconds...')

Reading all fif files for subject A01 # # # # # # # # # # # 
Reading all fif files for subject A02 # # # # # # # # # # # 
Reading all fif files for subject A03 # # # # # # # # # # # 
Reading all fif files for subject A04 # # # # # # # # # # # 
Reading all fif files for subject A05 # # # # # # # # # # # 
Reading all fif files for subject A06 # # # # # # # # # # # 
Reading all fif files for subject A07 # # # # # # # # # # # 
Reading all fif files for subject A08 # # # # # # # # # # # 
Reading all fif files for subject A09 # # # # # # # # # # # 
Reading all fif files for subject A10 # # # # # # # # # # # 
Finished concatenating, took me 42 seconds...


# 5. Filter  the data (HP 0.4 Hz and Notch 50 Hz)

In [ ]:
# Iterate over each subject and load the raw files:
start = time.time()
for subject, path in zip(subjects, paths):
    print(f'Reading raw files for subject {subject}', end=' ')
    file_names = [f for f in listdir(path) if '_unprocessed_raw.fif' in f]

    for i, file_name in enumerate(file_names):
        print(f'#', end=' ')

        file = path + '/' + file_name
        raw = mne.io.read_raw(file, preload=True)
        if plot:
            raw.plot(duration=60, proj=False, n_channels=len(raw.ch_names),
                     remove_dc=False, title='Highpass filtered')
            plot_spectrum(raw)


        # Highpass filter:
        raw_highpass = raw.copy().filter(l_freq=0.4, h_freq=None, picks=['eeg'], method='iir')
        if plot:
            raw_highpass.plot(duration=60, proj=False, n_channels=len(raw.ch_names),
                              remove_dc=False, title='Highpass filtered')
            plot_spectrum(raw_highpass)

        # Notch filter:
        raw_notch = raw_highpass.copy().notch_filter(freqs=[50], picks=['eeg'])
        if plot:
            raw_notch.plot(duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False, title='Notch filtered')
            plot_spectrum(raw_notch)

        # Store the raw file:
        store_name = path + '/' + subject + '_run_' + str(i + 1) + '_highpass_notch_filtered_raw.fif'
        raw_notch.save(store_name, overwrite=True)

    print()

print(f'Finished highpass and notch filtering, took me {round(time.time() - start)} seconds...')